Connected to .venv39 (3.9.22) (Python 3.9.22)

In [ ]:
# with the help of this file we can import the modules from Thesis folder

"""
Importing necessary libraries
"""

import os
import torch
import numpy as np
import pandas as pd
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import arrow
from datetime import datetime


ROOT = Path(__file__).resolve().parents[2]  # tests/ -> project root
sys.path.insert(0, str(ROOT))
print(ROOT)
# -------------************---------------------------------
# loading all the data loaders

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Modis_data_loader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import PM_25_dataloader
from notebooks.Modis_data_analysis.PM25_data_loader_analysis import (
    combine_the_data_frames,
)
from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Training_data_loader
from locationencoder.final_location_encoder import Geospatial_Encoder
from notebooks.Modis_data_analysis.torch_data_loader import AirQualityDataset

# importing config files
from notebooks.Modis_data_analysis.utils import config
from torch.utils.data import DataLoader

from notebooks.Modis_data_analysis.torch_data_loader import AirQualityDataset_latlon_AOD

from notebooks.Modis_data_analysis.torch_data_loader import AirQualityDataset_latlon_PM25

/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis


/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis/.venv39/lib/python3.9/site-packages/lightning/fabric/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)


/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
{'dataset_num_1': 1, 'dataset_num_2': 2, 'dataset_num_3': 3, 'exp_name': 'aod_pm25_prediction', 'input_type_1': 1, 'input_type_2': 2, 'input_type_3': 3, 'target_type': ['AOD', 'PM2.5'], 'input_vars': {1: ['latitude', 'longitude', 'AOD', 'PM2.5'], 2: ['latitude', 'longitude', 'AOD'], 3: ['latitude', 'longitude', 'PM2.5'], 4: ['latitude', 'longitude']}}
/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
{'dataset_num_1': 1, 'dataset_num_2': 2, 'dataset_num_3': 3, 'exp_name': 'aod_pm25_prediction', 'input_type_1': 1, 'input_type_2': 2, 'input_type_3': 3, 'target_type': ['AOD', 'PM2.5'], 'input_vars': {1: ['latitude', 'longitude', 'AOD', 'PM2.5'], 2: ['latitude', 'longitude', 'AOD'], 3: ['latitude', 'longitude', 'PM2.5'], 4: ['latitude', 'longitude']}}


In [ ]:
class Final_Air_Quality_Dataset_pipeline_latlon_AOD_PM25:
    def __init__(self, config):
        self.config = config
        self.Modis_data_2020 = None
        self.Modis_data_2021 = None
        self.Modis_data_2022 = None
        self.Modeis_data_2022 = None
        self.Modis_data_2023 = None
        self.stations = None
        self.PM_data = None
        self.Training_data = None
        self.torch_data = None
        self.geospatial_encoder_instance = Geospatial_Encoder(
            config["Geo_spatial_Encoder"]["dim_in"],
            config["Geo_spatial_Encoder"]["dim_hidden"],
            config["Geo_spatial_Encoder"]["dim_out"],
            config["Geo_spatial_Encoder"]["num_layers"],
        )

    def modis_data_sets(self):
        self.Modis_data_2018 = Modis_data_loader(
            "MODIS_AOD/merged_data_2018_sorted_dates.csv"
        )
        self.Modis_data_2019 = Modis_data_loader(
            "MODIS_AOD/merged_data_2019_sorted_dates.csv"
        )
        self.Modis_data_2020 = Modis_data_loader(
            "MODIS_AOD/merged_data_2020_sorted_dates.csv"
        )
        self.Modis_data_2021 = Modis_data_loader(
            "MODIS_AOD/merged_data_2021_sorted_dates.csv"
        )
        self.Modis_data_2022 = Modis_data_loader(
            "MODIS_AOD/merged_data_2022_sorted_dates.csv"
        )
        self.Modis_data_2023 = Modis_data_loader(
            "MODIS_AOD/merged_data_2023_sorted_dates.csv"
        )

        return [
            self.Modis_data_2020.data,
            self.Modis_data_2021.data,
            self.Modis_data_2022.data,
        ]

    def stations_data_sets(self):
        Lat_Long_data = Modis_data_loader("Stations_Lat_Long/stn_extrafeat.csv")
        self.stations = Lat_Long_data.data[["Latitude", "Longitude"]]
        return self.stations

    def PM_25_data(self):
        PM_data = PM_25_dataloader()
        self.PM_data = PM_data.get_data()
        return self.PM_data

    def training_data(self):
        loader = Training_data_loader(
            self.Modis_data_2020.data,
            self.Modis_data_2021.data,
            self.Modis_data_2022.data,
            self.PM_data,
            self.stations,
        )
        self.Training_data = loader.prepare_final_trainig_data()
        return self.Training_data

    def Torch_data(self):
        self.torch_data = AirQualityDataset(self.Training_data, self.config)
        return self.torch_data

    def full_pipeline(self):
        torch_latlong_loader = DataLoader(
            self.torch_data, batch_size=len(self.torch_data)
        )
        for inputs, outputs in torch_latlong_loader:
            all_latlong = inputs[:, 0:2]
        all_latitude_longitude_embeddings = self.geospatial_encoder_instance(
            all_latlong
        )
        final_inputs = torch.cat(
            (all_latitude_longitude_embeddings, inputs[:, 2:]), dim=1
        )

        return final_inputs, outputs

In [ ]:
# with the help of this file we can import the modules from Thesis folder

"""
Importing necessary libraries
"""

import os
import torch
import numpy as np
import pandas as pd
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import arrow
from datetime import datetime


ROOT = Path(__file__).resolve().parents[2]  # tests/ -> project root
sys.path.insert(0, str(ROOT))
print(ROOT)
# -------------************---------------------------------
# loading all the data loaders

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Modis_data_loader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import PM_25_dataloader
from notebooks.Modis_data_analysis.PM25_data_loader_analysis import (
    combine_the_data_frames,
)
from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Training_data_loader
from locationencoder.final_location_encoder import Geospatial_Encoder
from notebooks.Modis_data_analysis.torch_data_loader import AirQualityDataset_latlon_AOD_PM25

/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis


ImportError: cannot import name 'AirQualityDataset_latlon_AOD_PM25' from 'notebooks.Modis_data_analysis.torch_data_loader' (/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis/notebooks/Modis_data_analysis/torch_data_loader.py)

In [ ]:
# with the help of this file we can import the modules from Thesis folder

"""
Importing necessary libraries
"""

import os
import torch
import numpy as np
import pandas as pd
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import arrow
from datetime import datetime


ROOT = Path(__file__).resolve().parents[2]  # tests/ -> project root
sys.path.insert(0, str(ROOT))
print(ROOT)
# -------------************---------------------------------
# loading all the data loaders

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Modis_data_loader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import PM_25_dataloader
from notebooks.Modis_data_analysis.PM25_data_loader_analysis import (
    combine_the_data_frames,
)
from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Training_data_loader
from locationencoder.final_location_encoder import Geospatial_Encoder
from notebooks.Modis_data_analysis.torch_data_loader import (
    AirQualityDataset_latlon_AOD_PM25,
)

# importing config files
from notebooks.Modis_data_analysis.utils import config
from torch.utils.data import DataLoader

from notebooks.Modis_data_analysis.torch_data_loader import AirQualityDataset_latlon_AOD

from notebooks.Modis_data_analysis.torch_data_loader import (
    AirQualityDataset_latlon_PM25,
)

/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis


ImportError: cannot import name 'AirQualityDataset_latlon_AOD_PM25' from 'notebooks.Modis_data_analysis.torch_data_loader' (/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis/notebooks/Modis_data_analysis/torch_data_loader.py)

In [ ]:
# with the help of this file we can import the modules from Thesis folder

"""
Importing necessary libraries
"""

import os
import torch
import numpy as np
import pandas as pd
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import arrow
from datetime import datetime


ROOT = Path(__file__).resolve().parents[2]  # tests/ -> project root
sys.path.insert(0, str(ROOT))
print(ROOT)
# -------------************---------------------------------
# loading all the data loaders

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Modis_data_loader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import PM_25_dataloader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import combine_the_data_frames

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Training_data_loader
from locationencoder.final_location_encoder import Geospatial_Encoder
from notebooks.Modis_data_analysis.torch_data_loader import AirQualityDataset_latlon_AOD_PM25

/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis


ImportError: cannot import name 'AirQualityDataset_latlon_AOD_PM25' from 'notebooks.Modis_data_analysis.torch_data_loader' (/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis/notebooks/Modis_data_analysis/torch_data_loader.py)

Connected to .venv39 (3.9.22) (Python 3.9.22)

In [ ]:
# with the help of this file we can import the modules from Thesis folder

"""
Importing necessary libraries
"""

import os
import torch
import numpy as np
import pandas as pd
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import arrow
from datetime import datetime


ROOT = Path(__file__).resolve().parents[2]  # tests/ -> project root
sys.path.insert(0, str(ROOT))
print(ROOT)
# -------------************---------------------------------
# loading all the data loaders

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Modis_data_loader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import PM_25_dataloader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import (
    combine_the_data_frames,
)

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Training_data_loader
from locationencoder.final_location_encoder import Geospatial_Encoder
from notebooks.Modis_data_analysis.torch_data_loader import (
    AirQualityDataset_latlon_AOD_PM25,
)


# importing config files
from notebooks.Modis_data_analysis.utils import config
from torch.utils.data import DataLoader

from notebooks.Modis_data_analysis.torch_data_loader import AirQualityDataset_latlon_AOD

from notebooks.Modis_data_analysis.torch_data_loader import (
    AirQualityDataset_latlon_PM25,AirQualityDataset_latlon_AOD_PM25
)

/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis


/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis/.venv39/lib/python3.9/site-packages/lightning/fabric/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)


/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
{'dataset_num_1': 1, 'dataset_num_2': 2, 'dataset_num_3': 3, 'exp_name': 'aod_pm25_prediction', 'input_type_1': 1, 'input_type_2': 2, 'input_type_3': 3, 'target_type': ['AOD', 'PM2.5'], 'input_vars': {1: ['latitude', 'longitude', 'AOD', 'PM2.5'], 2: ['latitude', 'longitude', 'AOD'], 3: ['latitude', 'longitude', 'PM2.5'], 4: ['latitude', 'longitude']}}
/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis
{'dataset_num_1': 1, 'dataset_num_2': 2, 'dataset_num_3': 3, 'exp_name': 'aod_pm25_prediction', 'input_type_1': 1, 'input_type_2': 2, 'input_type_3': 3, 'target_type': ['AOD', 'PM2.5'], 'input_vars': {1: ['latitude', 'longitude', 'AOD', 'PM2.5'], 2: ['latitude', 'longitude', 'AOD'], 3: ['latitude', 'longitude', 'PM2.5'], 4: ['latitude', 'longitude']}}


In [ ]:
# with the help of this file we can import the modules from Thesis folder

"""
Importing necessary libraries
"""

import os
import torch
import numpy as np
import pandas as pd
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import arrow
from datetime import datetime


ROOT = Path(__file__).resolve().parents[2]  # tests/ -> project root
sys.path.insert(0, str(ROOT))
print(ROOT)
# -------------************---------------------------------
# loading all the data loaders

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Modis_data_loader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import PM_25_dataloader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import (
    combine_the_data_frames,
)

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Training_data_loader
from locationencoder.final_location_encoder import Geospatial_Encoder
from notebooks.Modis_data_analysis.torch_data_loader import (
    AirQualityDataset_latlon_AOD_PM25, AirQualityDataset_latlon_PM25, AirQualityDataset_latlon_AOD
)


# importing config files
from notebooks.Modis_data_analysis.utils import config
from torch.utils.data import DataLoader

/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis


In [ ]:
# with the help of this file we can import the modules from Thesis folder

"""
Importing necessary libraries
"""

import os
import torch
import numpy as np
import pandas as pd
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import arrow
from datetime import datetime


ROOT = Path(__file__).resolve().parents[2]  # tests/ -> project root
sys.path.insert(0, str(ROOT))
print(ROOT)
# -------------************---------------------------------
# loading all the data loaders

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Modis_data_loader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import PM_25_dataloader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import (
    combine_the_data_frames,
)

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Training_data_loader
from locationencoder.final_location_encoder import Geospatial_Encoder
from notebooks.Modis_data_analysis.torch_data_loader import (
    AirQualityDataset_latlon_AOD_PM25,
    AirQualityDataset_latlon_PM25,
    AirQualityDataset_latlon_AOD,AirQualityDataset_latlon
)

/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis


ImportError: cannot import name 'AirQualityDataset_latlon' from 'notebooks.Modis_data_analysis.torch_data_loader' (/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis/notebooks/Modis_data_analysis/torch_data_loader.py)

In [ ]:
# with the help of this file we can import the modules from Thesis folder

"""
Importing necessary libraries
"""

import os
import torch
import numpy as np
import pandas as pd
import sys
from pathlib import Path
import os
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
import arrow
from datetime import datetime


ROOT = Path(__file__).resolve().parents[2]  # tests/ -> project root
sys.path.insert(0, str(ROOT))
print(ROOT)
# -------------************---------------------------------
# loading all the data loaders

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Modis_data_loader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import PM_25_dataloader

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import (
    combine_the_data_frames,
)

from notebooks.Modis_data_analysis.PM25_data_loader_analysis import Training_data_loader
from locationencoder.final_location_encoder import Geospatial_Encoder
from notebooks.Modis_data_analysis.torch_data_loader import (
    AirQualityDataset_latlon_AOD_PM25,
    AirQualityDataset_latlon_PM25,
    AirQualityDataset_latlon_AOD,
    AirQualityDataset_latlon,
)

/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis


ImportError: cannot import name 'AirQualityDataset_latlon' from 'notebooks.Modis_data_analysis.torch_data_loader' (/Users/pavankumar/Documents/Winter_Thesis/Coding_Learning/Master-Thesis/notebooks/Modis_data_analysis/torch_data_loader.py)